In [ ]:
!pip install langgraph langchain_core langchain_groq langchain_huggingface langchain_chroma

In [3]:
from typing import TypedDict,Literal,Annotated,List
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
import os
from google.colab import userdata,drive
drive.mount('/content/drive')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
from langgraph.graph.message import add_messages
from langgraph.prebuilt import  ToolNode
from pydantic import  BaseModel,Field
from langchain_huggingface import HuggingFaceEmbeddings
from google.colab import drive
from langchain_chroma import Chroma
from langchain.retrievers import MultiQueryRetriever
from langchain_core.runnables import RunnableLambda, RunnablePassthrough,RunnableParallel
from langchain.tools import tool
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate

Mounted at /content/drive


# **Data Extractor and Formatter**

In [125]:
class Graphstate(TypedDict):
    user_name:str
    topic:str
    input_query:str
    due_date:str
    validation:Literal['valid','invalid']
    messages:str

class LLMOutput(BaseModel):
    user_name:str=Field(description='EmailId or Username')
    topic:str=Field(description='Topic or main content of user query')
    due_date:str=Field(description=('Date mentioned in user query'))

class Dataverify(BaseModel):
  output:str

def data_extractor(state:Graphstate):
    llm=ChatGroq(model='llama-3.1-8b-instant').with_structured_output(LLMOutput)

    prompt=f"""Kindly extract below mentioned data from user input message{state['input_query']}
              1)User name or User Email id
              2)Topic
              3)Date(MM/DD/YYYY)
              Provide your output in json format"""

    response=llm.invoke(prompt)

    return {'user_name':response.user_name,
            'topic':response.topic,
            'due_date':response.due_date,
            'messages':'Our query is being processed. We have prepared your data'}


def store_data(state:Graphstate):
  import pandas as pd
  received_info=pd.DataFrame(columns=['Username','Date','Topic'])
  received_info.loc[0,'Username']=state.get('user_name')
  received_info.loc[0,'Date']=state.get('due_date')
  received_info.loc[0,'Topic']=state.get('topic')
  return {'messages':f'Your necessary informations are stored {received_info}. Mail has sent to your mailbox '}

def query_verifier(state:Graphstate):
  llm=ChatGroq(model='llama-3.1-8b-instant').with_structured_output(Dataverify)
  prompt=f"""Check whether below informations are available in {state['input_query']}
             1)User Data
             2)Email ID
             Give your answer in JSON format with a key 'output' and a value of 'yes' or 'no'
             """

  validation=llm.invoke(prompt)

  return {'validation':validation.output,
          'messages':'Necessary informations are available'}


def user_redirecting(state:Graphstate):
  return {'messages':'Email ID or Date not available. Kindly provide it'}


def router(state:Graphstate):

  result=state.get('validation')
  if result.lower()=='yes':
    return 'valid'
  else:
    return 'invalid'


graph=StateGraph(Graphstate)

graph.add_node('query_verify',query_verifier)
graph.add_node('input_redirecting',user_redirecting)
graph.add_node('data_extractor',data_extractor)
graph.add_node('sending_data',store_data)

graph.set_entry_point('query_verify')
graph.add_conditional_edges('query_verify',
                            router,
                            {'valid':'data_extractor',
                             'invalid':'input_redirecting'}
                            )
graph.add_edge('data_extractor','sending_data')
graph.add_edge('sending_data',END)
graph.add_edge('input_redirecting',END)

app=graph.compile()





In [131]:
output=app.invoke({'input_query':'My name is Sam@gmail.com and I need data for Sales data for l/12/2024'})

In [132]:
output

{'user_name': 'Sam@gmail.com',
 'topic': 'Sales data',
 'input_query': 'My name is Sam@gmail.com and I need data for Sales data for l/12/2024',
 'due_date': '01/12/2024',
 'validation': 'yes',
 'messages': 'Your necessary informations are stored         Username        Date       Topic\n0  Sam@gmail.com  01/12/2024  Sales data. Mail has sent to your mailbox '}

#Human Looped Chatbot

In [9]:
def get_ra_agent():
  def format_docs(docs):
        return "\n\n".join(content.page_content for content in docs )
  db_persist_directory = "/content/drive/MyDrive/chroma_db_let_us_C"
  drive_base_path='/content/drive/MyDrive'
  hf_models_folder = 'HuggingFace_Models'
  model_name_to_save = 'all-MiniLM-L6-v2'
  hf_model_load_path = os.path.join(drive_base_path, hf_models_folder, model_name_to_save)

  embedding_instance=HuggingFaceEmbeddings(model_name=model_name_to_save)
  chroma_db=Chroma(embedding_function=embedding_instance,persist_directory=db_persist_directory)
  retriever=chroma_db.as_retriever(kwargs={'k':3})
  llm=ChatGroq(model='llama-3.1-8b-instant')
  multi_retriever=MultiQueryRetriever.from_llm(retriever=retriever,llm=llm)

  return multi_retriever
class Agent(TypedDict):
  question:str
  query_type:str
  response:Annotated[List[HumanMessage],add_messages]

@tool
def retrieve_information(question) ->str:
    "Retrieve necessary docs from vectorstore for user query and reframe it to give proper answer"

    multi_retriever=get_ra_agent()

    output=multi_retriever.invoke(question)

    print(f'No of documents retrieved is {len(output)}')

    return'\n\n'.join([doc.page_content for doc in output])

def retriever(agent:Agent):
    context=retrieve_information.invoke({'question':agent['question']})
    prompt=PromptTemplate(input_varibles=['question','context'],
                          template='You are an RAG based assistant who will be helpful in answering user query{query} with hel pf retrieved context{context}')
    llm=ChatGroq(model='llama-3.1-8b-instant')
    answer=llm.invoke(prompt.format(query=agent['question'],context=context))
    return {'response':answer}

def router(agent:Agent):

    llm=ChatGroq(model='llama-3.1-8b-instant')

    query_type=llm.invoke(f"""Classify whether user query is question or answer .
                            user_query - {agent['question']}
                            Kindly give output only in one word 'question' or 'end'   """).content
    if query_type.lower()=='question' or 'question' in query_type.lower():
      return {'query_type':'question'}
    else:
      return{'query_type':'end'}



graph=StateGraph(Agent)

graph.add_node('retriever',retriever)
graph.add_node('router',router)

graph.set_entry_point('router')
graph.add_conditional_edges('router',
                            lambda agent : 'question' if agent['query_type'] not in ['farewell','end','exit'] else 'end',
                            {'question':'retriever',
                             'end':END})
graph.add_edge('retriever',END)

app=graph.compile()





In [10]:
answer=app.invoke({'question':'What is functions'})

No of documents retrieved is 12


In [11]:
for m in answer['response']:
  m.pretty_print()

================================== Ai Message ==================================

Based on the provided text, I will provide an explanation of functions with the help of the retrieved context.

**What is a Function?**

A function is a self-contained block of statements that perform a coherent task of some kind. It is a way to isolate code that is related to a specific task, making it easier to reuse and maintain. Every C program can be thought of as a collection of these functions.

**Characteristics of Functions**

1. **Self-contained**: A function is a block of code that performs a specific task and can be used independently.
2. **Coherent task**: A function performs a single, well-defined task.
3. **Reusable**: Functions can be reused throughout a program.
4. **Maintainable**: Functions are easier to maintain and modify than a large block of code.

**Declaring and Defining Functions**

1. **Function declaration**: Specifies the return type of the function and the types of parameters

In [149]:
ag=get_ra_agent()

In [150]:
ag.invoke('What is pointers')

[]